In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import math
from IPython.display import HTML
%run Plot\ Utilities.ipynb

In [2]:
# Automate selection of step size ??

In [3]:
class GradientDescentAnim:
    
    def __init__(self, fn, xlim, x):
        self.fn = fn
        self.xlim = xlim
        self.x0 = x
        self.step_size = 0.1
        self.steps = 50
        self.walk = []
        
        
    def descend(self):
        delta = (self.xlim[1] - self.xlim[0]) / 20
        x = self.x0
        for i in range(self.steps):
            y = self.fn(x)
            g = 0.5 * (self.fn(x + delta) - self.fn(x - delta)) / delta
            self.walk.append((x, y, g))
            x -= self.step_size * g
            
    
    def drop_line(self, i):
        return [self.walk[i][0], self.walk[i][0]], [0, self.walk[i][1]]

    
    def tangent_line(self, i):
        g = self.walk[i][2]
        xlim = self.ax.get_xlim()
        ylim = self.ax.get_ylim()
        
        if abs(g) > (ylim[1] - ylim[0]) / (xlim[1] - xlim[0]):
            dy = (ylim[1] - ylim[0]) / 10
            dx = dy / g
        else:
            dx = (xlim[1] - xlim[0]) / 10
            dy = g * dx
        
        x = self.walk[i][0]
        y = self.walk[i][1]
        
        return [[x - dx, x + dx], [y - dy, y + dy]]
        

    def update(self, i):
        self.dl.set_data(*self.drop_line(i))
        self.tl.set_data(*self.tangent_line(i))
        
        
    def animate(self):
        self.descend()
        
        x = np.linspace(self.xlim[0], self.xlim[1])
        y = list(map(self.fn, x))
        self.fig, self.ax = plt.subplots()
        self.ax.plot(x, y, color = 'blue')
        
        plt.xlabel('w')
        plt.ylabel('Loss')
        self.dl, = self.ax.plot(*self.drop_line(0), color = 'black')
        self.tl, = self.ax.plot(*self.tangent_line(0), color = 'black')
        
        return matplotlib.animation.FuncAnimation(self.fig, lambda i: self.update(i), frames = len(self.walk))

In [4]:
class PerceptronAnim:
    
    def __init__(self, df, weights, label = 'y', trail = True):
        self.df = df
        self.weights = weights if type(weights) == list else list(weights)
        self.label = label
        self.features = list(df.columns)
        self.features.remove(label)
        self.trail = trail
                

    def bbox_point(self, x, y, m, b):
        if y is None:
            y = m * x + b
            ylim = self.ax.get_ylim()
            if y >= ylim[0] and y <= ylim[1]:
                return (x, y)
        if x is None:
            m = m or 0.001
            x = (y - b) / m
            xlim = self.ax.get_xlim()
            if x >= xlim[0] and x <= xlim[1]:
                return (x, y)
        return None

        
    def update(self, i):
        
        if i % 2 == 0:
            i = math.trunc(i / 2) % len(self.df)
            x = self.df.iloc[i][self.features][-2:]
            self.sc.set_offsets(([x.iloc[0], x.iloc[1]]))

        else:
            i = math.trunc(i / 2)
            m, b = weight_to_mb(self.weights[i])
            
            pts = [self.bbox_point(None, self.ax.get_ylim()[0], m, b)]
            pts.append(self.bbox_point(None, self.ax.get_ylim()[1], m, b))
            pts.append(self.bbox_point(self.ax.get_xlim()[0], None, m, b))
            pts.append(self.bbox_point(self.ax.get_xlim()[1], None, m, b))
            pts = [pt for pt in pts if pt]
            
            x = [a[0] for a in pts]
            y = [a[1] for a in pts]
            norm = np.linalg.norm(self.weights[i][-2:])
            if norm == 0:
                self.hp.set_data([], [])
            else:
                self.hp.set_data(x, y)
                        
            if self.trail and not np.array_equal(self.weights[i], self.weights[i - 1]):
                norm = np.linalg.norm(self.weights[i][-2:])
                if norm != 0 and len(x) > 0:
                    n = self.weights[i][-2:] / norm
                    self.ax.quiver([np.mean(x)], [np.mean(y)], [n.iloc[0] * 10], [n.iloc[1] * 10], color = 'black')
                
        
    def animate(self):        
        plot_2d(self.df, self.label, pad = 0.5)
        self.fig = plt.gcf()
        self.ax = plt.gca()

        self.hp, = self.ax.plot([0], [0], color = 'black')
        self.sc = self.ax.scatter([0], [0], s = 100, marker = 'o', color = 'black')
        
        return matplotlib.animation.FuncAnimation(self.fig, lambda i: self.update(i), frames = 2 * len(self.weights))

In [4]:
class LogisticRegressionAnim:
    
    def __init__(self, df, weights, label = 'y', trail = True, dot = False):
        self.df = df
        self.weights = weights if type(weights) == list else list(weights)
        self.label = label
        self.features = list(df.columns)
        self.features.remove(label)
        self.trail = trail
        self.scatters = None
        self.dot = dot
                

    def bbox_point(self, x, y, m, b):
        if y is None:
            y = m * x + b
            ylim = self.ax.get_ylim()
            if y >= ylim[0] and y <= ylim[1]:
                return (x, y)
        if x is None:
            m = m or 0.001
            x = (y - b) / m
            xlim = self.ax.get_xlim()
            if x >= xlim[0] and x <= xlim[1]:
                return (x, y)
        return None

        
    def update(self, i):
        
        if i % 2 == 0:
            if self.dot:
                i = math.trunc(i / 2) % len(self.df)
                x = self.df.iloc[i][self.features][-2:]
                self.sc.set_offsets(([x.iloc[0], x.iloc[1]]))

        else:
            i = math.trunc(i / 2)
            m, b = weight_to_mb(self.weights[i])
            
            pts = [self.bbox_point(None, self.ax.get_ylim()[0], m, b)]
            pts.append(self.bbox_point(None, self.ax.get_ylim()[1], m, b))
            pts.append(self.bbox_point(self.ax.get_xlim()[0], None, m, b))
            pts.append(self.bbox_point(self.ax.get_xlim()[1], None, m, b))
            pts = [pt for pt in pts if pt]
            
            x = [a[0] for a in pts]
            y = [a[1] for a in pts]
            norm = np.linalg.norm(self.weights[i][-2:])
            if norm == 0:
                self.hp.set_data([], [])
            else:
                self.hp.set_data(x, y)
                        
            if self.trail and not np.array_equal(self.weights[i], self.weights[i - 1]):
                norm = np.linalg.norm(self.weights[i][-2:])
                if norm != 0 and len(x) > 0:
                    n = self.weights[i][-2:] / norm
                    self.ax.quiver([np.mean(x)], [np.mean(y)], [n[0] * 10], [n[1] * 10], color = 'black')
                    
            for key in self.scatters:
                df2 = self.df[self.df[self.label] == key]
                p = df2[['x1', 'x2']].apply(lambda row: sigmoid(np.dot(row, self.weights[i].iloc[1:]) + self.weights[i].iloc[0]), axis = 1).values
                if key == 0:
                    p = 1 - p
                self.scatters[key].set_array(p)
                
        
    def animate(self):        
        self.scatters = plot_2d(self.df, self.label, pad = 0.5)
        self.fig = plt.gcf()
        self.ax = plt.gca()
        
        self.hp, = self.ax.plot([0], [0], color = 'black')
        if self.dot:
            self.sc = self.ax.scatter([0], [0], s = 100, marker = 'o', color = 'black')
        else:
            self.sc = None
        
        return matplotlib.animation.FuncAnimation(self.fig, lambda i: self.update(i), frames = 2 * len(self.weights))